In [1]:
# Goal is to create python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund 

In [2]:
import numpy as np
import pandas as pd 
import requests
# request is gold standard for making http requests. Http request is basically internet request that you can make to an API to get some data
import xlsxwriter
import math

In [3]:
#Now we import the constitutents of the S&P 500 

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [5]:
# Acquire an API Token...import our IEX cloud API token.
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
# Make first API call
# base url is a url that will start every http request and after that you have to add which specific endpoint that you want to retrieve from the API,
# most api expose certain data through each endpoint which makes things faster if you only need to retrieve certain data 

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'LGRNE M /GTSAOAAL)BQDEELAS(KSTC N', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'oifcfial', 'close': None, 'closeTime': None, 'closeSource': 'lofaicfi', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 133.32, 'latestSource': 'IEX real time price', 'latestTime': '3:09:50 PM', 'latestUpdate': 1690334080951, 'latestVolume': None, 'iexRealtimePrice': 131.18, 'iexRealtimeSize': 99, 'iexLastUpdated': 1643207088745, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 136.8, 'previousVolume': 98429980, 'change': -3.2, 'changePercent': -0.02407, 'volume': None, 'iexMarketPercent': 0.01500837350004694, 'iexVolume': 1735055, 'avgTotalVolume': 112337799, 'iexBi

In [10]:
# now parse
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)

2.202381575744


In [17]:
#Add stocks data to pandas dataframe 
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [22]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A' #start with NA now and then add it later
        ],
    index = my_columns),
   
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,133.32,2202381575744,N/A


In [27]:
#loop through tickers in sp500 list of stocks 
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True
    )

In [28]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.51,38553590898,N/A
1,AAL,16.00,9835496169,N/A
2,AAP,174.28,11415550358,N/A
3,AAPL,130.60,2177270529040,N/A
4,ABBV,110.72,194064490692,N/A
...,...,...,...,...
500,YUM,108.36,33611094753,N/A
501,ZBH,167.80,33292340405,N/A
502,ZBRA,398.52,21588549866,N/A
503,ZION,48.56,8184748219,N/A


In [29]:
# using batch API Calls to improve performance. 
# Batch API calls are one of the easiest way to improve performance of the code. This is becaseu HTTP requests are typically one of the slowest components of a script 
# Also API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to 
# IEX cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls
def chunks(lst, n):
    """Yield successive n sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        

In [31]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
for i in range(0, len(symbol_groups)):
    print(symbol_groups[i])

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object
100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object
200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object
300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object
400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, dtype: object
